In [1]:
!pip install razdel -q

# Домашнее задание № 4. Языковые модели

In [2]:
from string import punctuation
from razdel import sentenize
from razdel import tokenize as razdel_tokenize
import numpy as np
from IPython.display import Image
from IPython.core.display import HTML
import nltk
nltk.download('punkt_tab')
from nltk.tokenize import sent_tokenize
from sklearn.model_selection import train_test_split
from collections import Counter

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


## Задание 1 (8 баллов).

В семинаре для генерации мы использовали предположение маркова и считали, что слово зависит только от 1 предыдущего слова. Но ничто нам не мешает попробовать увеличить размер окна и учитывать два или даже три прошлых слова. Для них мы еще сможем собрать достаточно статистик и, логично предположить, что качество сгенерированного текста должно вырасти.

Попробуйте сделать языковую модель, которая будет учитывать два предыдущих слова при генерации текста.
Сгенерируйте несколько текстов (3-5) и расчитайте перплексию получившейся модели.
Можно использовать данные из семинара или любые другие (можно брать только часть текста, если считается слишком долго). Перплексию рассчитывайте на 10-50 отложенных предложениях (они не должны использоваться при сборе статистик).


Подсказки:  
    - нужно будет добавить еще один тэг \<start>  
    - можете использовать тот же подход с матрицей вероятностей, но по строкам хронить биграмы, а по колонкам униграммы
    - тексты должны быть очень похожи на нормальные (если у вас получается рандомная каша, вы что-то делаете не так)
    - у вас будут словари с индексами биграммов и униграммов, не перепутайте их при переводе индекса в слово - словарь биграммов будет больше словаря униграммов и все индексы из униграммного словаря будут формально подходить для словаря биграммов (не будет ошибки при id2bigram[unigram_id]), но маппинг при этом будет совершенно неправильным

Чтение файла

In [3]:
news = open("lenta.txt").read()

Нормализация:

In [4]:
def normalize(text):
    normalized_text = [word.text.strip(punctuation) for word \
                                                            in razdel_tokenize(text)]
    normalized_text = [word.lower() for word in normalized_text if word and len(word) < 20 ]
    return normalized_text

In [5]:
def ngrammer(tokens, n):
    ngrams = []
    for i in range(0,len(tokens)-n+1):
        ngrams.append(' '.join(tokens[i:i+n]))
    return ngrams

In [6]:
sentences_news = [['<start>'] + ['<start>'] + normalize(text) + ['<end>'] for text in sent_tokenize(news[:500000])]

In [7]:
sents_news_train, sents_news_val = train_test_split(sentences_news, test_size=50, random_state=42)

In [8]:
unigrams_news = Counter()
bigrams_news = Counter()
trigrams_news = Counter()

for sentence in sentences_news:
    unigrams_news.update(sentence)
    bigrams_news.update(ngrammer(sentence, n=2))
    trigrams_news.update(ngrammer(sentence, n=3))

In [9]:
from scipy.sparse import lil_matrix, csr_matrix, csc_matrix

In [61]:
# делаем то же, что на семинаре, но теперь для биграмм и униграмм
matrix_news = lil_matrix((len(bigrams_news),
                        len(unigrams_news)))

id2word_news = list(unigrams_news)
word2id_news = {word:i for i, word in enumerate(id2word_news)}

id2bigram_news = list(bigrams_news)
bigram2id_news = {bigram:i for i, bigram in enumerate(id2bigram_news)}

for ngram in trigrams_news:
    bigram, word = ngram.rsplit(" ", 1)
    matrix_news[bigram2id_news[bigram], word2id_news[word]] =  (trigrams_news[ngram]/
                                                                     bigrams_news[bigram])

matrix_news = csc_matrix(matrix_news)

In [100]:
def generate(matrix, id2word, word2id, id2bigram, bigram2id, n=100, start="<start> <start>"):
    text = []
    current_idx = bigram2id[start]
    prev_word=["<start>"]

    for i in range(n):

        chosen = np.random.choice(matrix.shape[1], p=matrix[current_idx].toarray()[0])
        text.append(id2word[chosen])

        if id2word[chosen] == '<end>':
            prev_word=["<start>"]
            chosen = word2id['<start>']

        prev_word.append(id2word[chosen])
        current_idx = bigram2id[" ".join(prev_word)]
        prev_word = prev_word[1:]

    return ' '.join(text)

In [101]:
from pprint import pprint

print(generate(matrix_news, id2word_news, word2id_news, id2bigram_news, bigram2id_news))

глава комитета <end> дом был оцеплен частями милиции на место оперативники задержали молодого человека который указал им место где он со своими зарубежными коллегами более опытным в противостоянии этому злу прежде всего неалбанской его части <end> входе перестрелки с правительственными войсками и сепаратистами из организации тигры освобождения тамил илама тоти происходят практически повсеместно на северо-востоке от голчука и всего в andava и forus <end> постановление принято в связи с террористическими актами в столице значительно увеличилось количество желающих безвозмездно сдать свою кровь для нужд пострадавших в результате 6 человек получили ножевые ранения в прибрежном городе голчук <end> выступая по местному времени


# Перплексия

In [107]:
def compute_joint_proba_markov_assumption(text, bigram_counts, trigram_counts):
    prob = 0
    for ngram in ngrammer(['<start>'] + normalize(text) + ['<end>'], 3):
        bigram, word = ngram.rsplit(" ", 1)
        if bigram in bigram_counts and ngram in trigram_counts:
            prob += np.log(trigram_counts[ngram]/bigram_counts[bigram])
        # small value for unk words
        else:
            prob += np.log(2e-5)

    return np.exp(prob)

In [ ]:
def compute_join_proba_markov_assumption(
    text, smaller_ngram_counts, ngram_counts,
    n=1, tokenized=True
):
    prob = 0
    # if tokenized:
    #     tokenized_text = text
    # else:
    #     tokenized_text = ["<start>"] * n + normalize(text) + ["<end>"]

    for ngram in ngrammer(tokenized_text, n=n+1):
        left_ngram, word = ngram.rsplit(" ", 1)
        if left_ngram in smaller_ngram_counts and ngram in ngram_counts:
            prob += np.log(
                ngram_counts[ngram] / smaller_ngram_counts[left_ngram]
            )
        else:
            prob += np.log(2e-5)
    return prob, len(tokenized_text) - n - 1

In [112]:
perplexity = []

for sent in sents_news_val:
    perplexity.append(compute_joint_proba_markov_assumption(" ".join(sent), bigrams_news, trigrams_news))

np.mean(perplexity)

np.float64(3.1285607395677934e-25)